# Ensemble Model

In [3]:
!pip install tensorflow==2.15.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 475.3/475.3 MB 1.6 MB/s eta 0:00:0000:0100:02
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 1.5 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 1.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.5/5.5 MB 1.2 MB/s eta 0:00:0000:0100:010m
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 442.0/442.0 kB 1.0 MB/s eta 0:00:0000:0100:01
  Attempting uninstall: ml-dtypes
    Found existing installation: ml-dtypes 0.3.2
    Uninstalling ml-dtypes-0.3.2:
      Successfully uninstalled ml-dtypes-0.3.2
  Attempting uninstall: keras
    Found existing installation: keras 3.1.1
    Uninstalling keras-3.1.1:
      Successfully uninstalled keras-3.1.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 2.16.2
    Uninstalling tensorboard-2.16.2:
      Successfully uninstalled tensorboard-2.16.2
  Attempting uninstall: tensorflow
    Found existing installation: ten

## 1. Imports

In [4]:
import os
import zipfile
import tensorflow as tf
import urllib.request as request

from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Input, Average

## 2. Load already trained model

In [5]:
model_mobilenet = load_model('./models/mobilenet_no_exp.keras')
model_mobilenet = Model(inputs=model_mobilenet.inputs,
                outputs=model_mobilenet.outputs,
                name='model_mobilenet')

model_mobilenet_v2 = load_model('./models/mobilenetv2_no_exp.keras')
model_mobilenet_v2 = Model(inputs=model_mobilenet_v2.inputs,
                outputs=model_mobilenet_v2.outputs,
                name='model_mobilenet')




/home/surajkarki/anaconda3/lib/python3.11/site-packages/keras/src/saving/saving_lib.py:396: UserWarning: Skipping variable loading for optimizer 'adam', because it has 168 variables whereas the saved optimizer has 6 variables. 
  trackable.load_own_variables(weights_store.get(inner_path))


ImportError: cannot import name 'disable_resource_variables' from 'tensorflow.python.ops.variable_scope' (/home/surajkarki/anaconda3/lib/python3.11/site-packages/tensorflow/python/ops/variable_scope.py)

## 3. Building an ensemble model

In [5]:
def ensemble(models, model_input):
    outputs = [model.outputs[0] for model in models]
    y = Average()(outputs)
    model = Model(model_input, y, name='ensemble')
    return model

In [6]:
model_input = Input(shape=(224, 224, 3))

ensemble_model = ensemble([model_nasnet_mobile, model_mobilenet], model_input)

ValueError: The name "input_layer" is used 2 times in the model. All operation names should be unique.

In [ ]:
ensemble_model.compile(optimizer="adam",
                       loss=tf.keras.losses.CategoricalCrossentropy(),
                       metrics=[
                           tf.keras.metrics.TopKCategoricalAccuracy(
                               name="top1_accuracy", k=1),
                           tf.keras.metrics.TopKCategoricalAccuracy(
                               name="top5_accuracy"),
                           tf.keras.metrics.Precision(name='precision'),
                           tf.keras.metrics.Recall(name='recall'),
                           tf.keras.metrics.AUC(
                               name="auc", multi_label=True, num_labels=30, from_logits=False, label_weights=None),

                       ]
                       )

## 4. Data Preparation

In [18]:
filename, headers = request.urlretrieve(
            url="https://gitlab.com/surajkarki66/medicinal_plant_leaf_images/-/raw/main/dataset_v2.zip?ref_type=heads",
            filename="./data/dataset_v2.zip",
        )
print(f"{filename} download! with following info: \n{headers}")

./data/dataset_v2.zip download! with following info: 
Date: Mon, 08 Jul 2024 17:55:24 GMT
Content-Type: application/zip
Content-Length: 694279746
Connection: close
cache-control: max-age=60, public, must-revalidate, stale-while-revalidate=60, stale-if-error=300, s-maxage=60
content-disposition: attachment; filename="dataset_v2.zip"; filename*=UTF-8''dataset_v2.zip
content-security-policy: base-uri 'self'; child-src https://www.google.com/recaptcha/ https://www.recaptcha.net/ https://www.googletagmanager.com/ns.html https://*.zuora.com/apps/PublicHostedPageLite.do https://gitlab.com/admin/ https://gitlab.com/assets/ https://gitlab.com/-/speedscope/index.html https://gitlab.com/-/sandbox/ 'self' https://gitlab.com/assets/ blob: data:; connect-src 'self' https://gitlab.com wss://gitlab.com https://sentry.gitlab.net https://new-sentry.gitlab.net https://customers.gitlab.com https://snowplow.trx.gitlab.net https://sourcegraph.com https://collector.prd-278964.gl-product-analytics.com snowplo

In [26]:
unzip_path = "./data/"
os.makedirs(unzip_path, exist_ok=True)
with zipfile.ZipFile("./data/dataset_v2.zip", 'r') as zip_ref:
    zip_ref.extractall(unzip_path)

In [27]:
BATCH_SIZE = 32

In [28]:
datagenerator_kwargs = dict(
    rescale=1./255,
)
dataflow_kwargs = dict(
    target_size=(224, 224, 3),
    batch_size=BATCH_SIZE,
    class_mode="categorical",
)

In [29]:
training_data_path = os.path.join("./data", "dataset_v2", "train")
validation_data_path = os.path.join("./data", "dataset_v2", "val")

In [30]:
valid_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
    **datagenerator_kwargs
)

train_datagenerator = tf.keras.preprocessing.image.ImageDataGenerator(
    rotation_range=40,
    horizontal_flip=True,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    **datagenerator_kwargs
)

train_generator = train_datagenerator.flow_from_directory(
    directory=training_data_path,
    shuffle=True,
    **dataflow_kwargs
)
valid_generator = valid_datagenerator.flow_from_directory(
    directory=validation_data_path,
    shuffle=False,
    **dataflow_kwargs
)

Found 4116 images belonging to 30 classes.
Found 509 images belonging to 30 classes.
